In [1]:
# GPU 사용가능 여부 확인
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-09-06 11:33:17.252368: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-09-06 11:33:17.272912: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3600000000 Hz
2022-09-06 11:33:17.273186: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cd422cfd40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-09-06 11:33:17.273218: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-09-06 11:33:17.273749: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-09-06 11:33:17.485254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11428544367718355772,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8028670431789947775
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1790705664
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8076823467314127678
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 3278569472
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3386998151761646866
 physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:02:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8439180103194919327
 physical_device_desc: "device: XLA_GPU de

In [2]:
#파이썬 버전 확인
import sys
print(sys.version)

3.7.11 (default, Jul 27 2021, 14:32:16) 
[GCC 7.5.0]


In [3]:
#텐서플로우 버전 확인
import tensorflow as tf
tf.__version__

'1.15.0'

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "/device:GPU:0"

In [5]:
# common library
import pandas as pd
import numpy as np
import time
from stable_baselines.common.vec_env import DummyVecEnv
# config
from config.config_tse import *
# preprocessor
from preprocessing.preprocessors_tse import * # from folder.file import *
# model
from model.models_tse import *

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/home/kongminseok/anaconda3/envs/k/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [6]:
def run_model() -> None:
    """Train the model."""

    # read and preprocess data

    preprocessed_path = "data/tse30_done_data.csv"
    if os.path.exists(preprocessed_path):
        data = pd.read_csv(preprocessed_path, index_col=0) # 0번째 column을 인덱스로 지정
    else:
        data = preprocess_data()
        data = add_turbulence(data)
        data.to_csv(preprocessed_path)

    # print(data.head())
    # print(data.size)

    # 2015/10/01 is the date that validation starts
    # 2016/01/01 is the date that real trading starts
    # unique_trade_date needs to start from 2015/10/01 for validation purpose
    
    unique_trade_date = data[(data.datadate > 20151001)].datadate.unique()
    #print(unique_trade_date)

    # rebalance_window is the number of months to retrain the model
    # validation_window is the number of months to validation the model and select for trading
    rebalance_window = 63 # default=63
    validation_window = 63 # default=63
    
    
    ## Original Ensemble Strategy
    #run_ensemble_strategy(df=data, 
    #                      unique_trade_date= unique_trade_date,
    #                      rebalance_window = rebalance_window,
    #                      validation_window= validation_window)
    
    ## Remake Ensemble
    #run_remake_ensemble(df=data, 
    #              unique_trade_date= unique_trade_date,
    #              rebalance_window = rebalance_window,
    #              validation_window= validation_window)
    
    ## Renewal Ensemble2
    #run_ensemble2(df=data, 
    #              unique_trade_date= unique_trade_date,
    #              rebalance_window = rebalance_window,
    #              validation_window= validation_window)
    

    ## PPO
    #run_ppo(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## A2C
    run_a2c(df=data, 
            unique_trade_date= unique_trade_date,
            rebalance_window = rebalance_window,
            validation_window= validation_window)
    
    ## DDPG
    #run_ddpg(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #       validation_window= validation_window)
    
    ## ACKTR
    #run_acktr(df=data, 
    #       unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #       validation_window= validation_window)
    
    ## TRPO
    #run_trpo(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## SAC
    #run_sac(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## TD3
    #run_td3(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## GAIL
    #run_gail(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)


#     #_logger.info(f"saving model version: {_version}")
if __name__ == "__main__":
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        print(result)
        run_model()

result_tse_a2c_HMAX_NORMALIZE10_hold0to3_5


2022-09-06 11:33:18.100707: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 11:33:18.100957: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:01:00.0
2022-09-06 11:33:18.100995: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 11:33:18.101180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:02:00.0
2022-09-06 11:33:18.101203: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic li

============Start A2C============
turbulence_threshold:  220.25731649343967
======Model training from:  20090000 to  20151002
======A2C Training========




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.



2022-09-06 11:33:18.222033: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 11:33:18.222345: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:01:00.0
2022-09-06 11:33:18.222411: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 11:33:18.222736: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:02:00.0
2022-09-06 11:33:18.222764: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic li





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




...

Training time (A2C):  9.82379051844279  minutes
======A2C Validation from:  20190712 to  20191016
A2C Sharpe Ratio:  0.24428344804476232
======Trading from:  20191016 to  20200122
previous_total_asset:154230495.6285379
end_total_asset:164995552.62452865
total_reward:10765056.995990753
total_cost:  135305.95619622813
total trades:  1638
Sharpe:  0.2955542198750044
============Trading Done============
turbulence_threshold:  220.25731649343967
======Model training from:  20090000 to  20191016
======A2C Training========


2022-09-06 14:10:50.834519: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 14:10:50.834719: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:01:00.0
2022-09-06 14:10:50.834768: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 14:10:50.834996: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:02:00.0
2022-09-06 14:10:50.835023: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic li

Training time (A2C):  9.902211586634317  minutes
======A2C Validation from:  20191016 to  20200122
A2C Sharpe Ratio:  -0.065769251650175
======Trading from:  20200122 to  20200423
previous_total_asset:164995552.62452865
end_total_asset:133320439.66805814
total_reward:-31675112.956470504
total_cost:  99688.7448008422
total trades:  1416
Sharpe:  -0.26190378429104016
============Trading Done============
turbulence_threshold:  220.25731649343967
======Model training from:  20090000 to  20200122
======A2C Training========


2022-09-06 14:20:46.214574: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 14:20:46.214774: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:01:00.0
2022-09-06 14:20:46.214811: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 14:20:46.215001: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.65
pciBusID: 0000:02:00.0
2022-09-06 14:20:46.215027: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic li

Training time (A2C):  9.781779686609903  minutes
======A2C Validation from:  20200122 to  20200423
A2C Sharpe Ratio:  0.05887526379701405
======Trading from:  20200423 to  20200729
previous_total_asset:133320439.66805814
end_total_asset:161003552.7237851
total_reward:27683113.05572696
total_cost:  116896.90279476927
total trades:  1523
Sharpe:  0.44632301586596546
============Trading Done============
A2C took:  177.2698863307635  minutes


-----------